In [ ]:
from __future__ import print_function

import os, sys
module_path = os.path.abspath(os.path.join('../../..'))
sys.path.append(module_path)

import numpy as np
import math
import copy
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist
import time
import pickle

from pycrcnn.he.HE import TFHEnuFHE
from pycrcnn.he.tfhe_value import TFHEValue
from pycrcnn.he.alu import *

In [ ]:
def calcScaleZeroPoint(min_val, max_val,num_bits):
  # Calc Scale and zero point
  qmin = -2.**(num_bits-1)
  qmax = 2.**(num_bits-1) - 1.

  scale = (max_val - min_val) / (qmax - qmin)

  initial_zero_point = qmin - min_val / scale
  
  zero_point = 0
  if initial_zero_point < qmin:
      zero_point = qmin
  elif initial_zero_point > qmax:
      zero_point = qmax
  else:
      zero_point = initial_zero_point

  zero_point = int(zero_point)

  return scale, zero_point

def quantize_tensor(x, num_bits, min_val=None, max_val=None):
    
    if not min_val and not max_val: 
      min_val, max_val = x.min(), x.max()

    qmin = -2.**(num_bits-1)
    qmax = 2.**(num_bits-1) - 1.

    scale, zero_point = calcScaleZeroPoint(min_val, max_val, num_bits)
    q_x = zero_point + x / scale
    q_x = q_x.clip(qmin, qmax)
    q_x = q_x.astype(float).round().astype(int)
    
    return q_x
  
def quantize_tensor2(x, num_bits, min_val=None, max_val=None):
    
    if not min_val and not max_val: 
        min_val, max_val = x.min(), x.max()

    qmin = -2.**(num_bits-1)
    qmax = 2.**(num_bits-1) - 1.

    
    x = x - min_val          # Allineo tutto l'array in modo che parta da 0
    x /= (max_val - min_val) # Lo scalo tra 0 e 1    
    x *= (qmax - qmin)       # Lo scalo tra 0 e 16
    x -= qmax                # Lo sfaso tra -8 e 7
    
    q_x = x.astype(float).round().astype(int)
    
    return q_x

In [ ]:
from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data
# reshape and normalize input data
x_train = x_train[:, 6:22, 6:22] # Crop f1 -> 6:22, f2 -> 2:26

train_indexes, test_indexes = [], []

# ternary classification
for i in range(len(x_train)):
  if y_train[i] == 0 or y_train[i] == 1 or y_train[i] == 2:
    train_indexes.append(i)

for i in range(len(x_test)):
  if y_test[i] == 0 or y_test[i] == 1 or y_test[i] == 2:
    test_indexes.append(i)

x_train = np.subtract(x_train[train_indexes], 128)
x_train.dtype = np.int8
y_train = y_train[train_indexes]

trainImages = 15000
x_train, x_val = x_train[:trainImages], x_train[trainImages:]
y_train, y_val = y_train[:trainImages], y_train[trainImages:]

# encode output which is a number in range [0,2] into a vector of size 3
# e.g. number 3 will become [0, 0, 16]
y_train = np_utils.to_categorical(y_train).astype(int)*16

# same for test data
x_test = x_test[:, 6:22, 6:22] # Crop f1 -> 6:22, f2 -> 2:26
x_test = np.subtract(x_test[test_indexes], 128)
x_test.dtype = np.int8
y_test = y_test[test_indexes]

In [ ]:
HE_Client = TFHEnuFHE(21)

with open("../secret_key", "rb") as f:
    HE_Client.secret_key = HE_Client.ctx.load_secret_key(f)
    
with open("../cloud_key", "rb") as f:
    HE_Client.cloud_key = HE_Client.ctx.load_cloud_key(f)

cloud_key = HE_Client.cloud_key
HE_Client.generate_vm(cloud_key)

In [ ]:
num1 = HE_Client.encrypt(1)
num2 = HE_Client.encode(6)
sum = num1+num2
mul = num1*num2

In [ ]:
SHRT_MAX = 32767
SHRT_MIN = (-SHRT_MAX - 1 )

def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [ ]:
def encrypted_pocketTanh(matIn, bits, inDims, outDims):
    yMax = HE_Client.encode(128)
    yMin = HE_Client.encode(-127)
    joints = HE_Client.encode_matrix([128, 75, 32, -31, -74, -127])
    divisor = (1 << bits) * inDims
    slopesInv = HE_Client.encode_matrix([128, 8, 2, 1, 2, 8, 128])

    matOut = np.full((matIn.shape[0], outDims), yMax)
    matActvGradInv = np.full((matIn.shape[0], outDims), slopesInv[0])

    for i in range(len(matIn)):
      for j in range(len(matIn[i].squeeze())):
        x = matIn[i].squeeze()[j] / divisor

        lt0 = x < joints[0]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt0, (x / 4).value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt0, slopesInv[1].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt1 = x < joints[1]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt1, x.value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt1, slopesInv[2].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt2 = x < joints[2]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt2, (x * 2).value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt2, slopesInv[3].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt3 = x < joints[3]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt3, x.value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt3, slopesInv[4].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt4 = x < joints[4]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt4, (x / 4).value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt4, slopesInv[5].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

        lt5 = x < joints[5]
        matOut[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt5, yMin.value, matOut[i][j].value), x.vm, x.n_bits)
        matActvGradInv[i][j] = TFHEValue(HE_Client.vm.gate_mux(lt5, slopesInv[6].value, matActvGradInv[i][j].value), x.vm, x.n_bits)

    return matOut, matActvGradInv

In [ ]:
def scalarL2LossDelta(y, yHat):
    return (yHat - y)

def batchL2LossDelta(yMat, yHatMat):
    # Assumption: 1 input -> 1 scalar sumLoss value
    # for 1 output of dimention T, lossDeltaMat = (1, T)
    lossDeltaMat = np.full((yMat.shape[0], yMat.shape[1]), HE_Client.encode(0))
    # Per each input item
    for i in range(len(yMat)):
      for j in range(len(yMat[i])):
        scalarLossDelta = scalarL2LossDelta(yMat[i][j], yHatMat[i].squeeze()[j])
        lossDeltaMat[i][j] = scalarLossDelta

    # return sum! (average is meaningless)
    return lossDeltaMat

In [ ]:
# inherit from base class Layer
class EncryptedMaxPoolLayer:
  
  def __init__(self, kernel_size, stride=(1, 1)):
    self.kernel_size = kernel_size
    self.stride = stride

  def forward(self, batch):
    return np.array([_max(image, self.kernel_size, self.stride) for image in batch])

  def backward(self, lastLayerDeltasMat, lrInv):
      return lastLayerDeltasMat

def _max(image, kernel_size, stride):
    x_s = stride[1]
    y_s = stride[0]

    x_k = kernel_size[1]
    y_k = kernel_size[0]

    # print(image)
    x_d = len(image[0])
    y_d = len(image)

    x_o = ((x_d - x_k) // x_s) + 1
    y_o = ((y_d - y_k) // y_s) + 1

    def get_submatrix(matrix, x, y):
        index_row = y * y_s
        index_column = x * x_s
        return matrix[index_row: index_row + y_k, index_column: index_column + x_k]

    return [[encrypted_max(get_submatrix(image, x, y).flatten()) for x in range(0, x_o)] for y in range(0, y_o)]

In [ ]:
class EncryptedFlattenLayer:

    def __init__(self):
        pass

    def forward(self, image):
        return image.reshape(image.shape[0], image.shape[1]*image.shape[2])

    def backward(self, lastLayerDeltasMat, lrInv):
      return lastLayerDeltasMat

In [ ]:
# inherit from base class Layer
class EncryptedFCLayer:
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size, outLayer = False):
      self.input_size = input_size
      self.output_size = output_size
      self.outLayer = outLayer
      self.weights = np.zeros((input_size, output_size)).astype(int)
      self.bias = HE_Client.encode_matrix(np.zeros((1, output_size)).astype(int))
      self.mDfaWeight = np.zeros((1, 1)).astype(int)

    # returns output for a given input
    def forward(self, input_data):
        self.input = input_data        
        dot = np.matmul(self.input, self.weights) + self.bias
        output, self.matActvGradInv = encrypted_pocketTanh(dot, 8, self.input_size, self.output_size)
        return output

    def backward(self, lastLayerDeltasMat, lrInv):      
      mDeltas = self.computeDeltas(lastLayerDeltasMat, lrInv)
      batchSize = len(mDeltas) # 1 for one item

      mWeightUpdate = np.matmul(self.input.T, mDeltas)
      mWeightUpdate = mWeightUpdate / lrInv
      mWeightUpdate = mWeightUpdate.reshape(self.input_size, self.output_size)

      if type(self.weights.squeeze()[0][0]) is not TFHEValue:
        self.weights = HE_Client.encode_matrix(self.weights)
        
      self.weights -= mWeightUpdate

      ones = np.ones((batchSize, 1)).astype(int)
      mBiasUpdate = np.matmul(mDeltas.T, ones)
      mBiasUpdate = mBiasUpdate.T / lrInv
      self.bias -= mBiasUpdate

    def setRandomDfaWeight(self, mInDim, mOutDim):
      range = isqrt((12 * SHRT_MAX) / (mInDim + mOutDim))
      self.mDfaWeight = np.random.randint(-range, range, (mInDim, mOutDim))
 
    def computeDeltas(self, lastLayerDeltasMat, lrInv):
      if self.outLayer:
        mDeltas = np.divide(lastLayerDeltasMat, self.matActvGradInv)
      else:
        if self.mDfaWeight.shape[0] != lastLayerDeltasMat.shape[1] and  self.mDfaWeight.shape[1] != self.weights.shape[1]: # 0 rows, 1 cols
          print("Initialized DFA")
          self.setRandomDfaWeight(lastLayerDeltasMat.shape[1], self.weights.shape[1])
        dot = np.matmul(lastLayerDeltasMat, self.mDfaWeight)   
        mDeltas = np.divide(dot, self.matActvGradInv)
      return mDeltas

In [ ]:
class EncryptedNetwork:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)
    
    # serialize the network
    def serialize(self):
        for l in self.layers:
          if hasattr(l, "weights"):
            l.weights = HE_Client.serialize_matrix(l.weights)
            l.bias = HE_Client.serialize_matrix(l.bias)
            l.matActvGradInv = None
            l.input = None
            if not l.outLayer:
              l.mDfaWeight = HE_Client.serialize_matrix(l.mDfaWeight)
    
    # deserialize the network
    def deserialize(self):
        for l in self.layers:
          if hasattr(l, "weights"):
            l.weights = HE_Client.deserialize_matrix(l.weights)
            l.bias = HE_Client.deserialize_matrix(l.bias)
            if not l.outLayer:
              l.mDfaWeight = HE_Client.deserialize_matrix(l.mDfaWeight)

    # test
    def test(self, x_test, y_test):
      # sample dimension first
      samples = len(x_test)
      corr = HE_Client.encode(0)
      enc_x = HE_Client.encrypt_matrix(x_test)
      enc_y = HE_Client.encrypt_matrix(y_test)

      for j in range(samples):
          # forward propagation
          pred = self.predict(enc_x[j])
          corr = TFHEValue(HE_Client.vm.gate_mux(pred == enc_y[j][0], (corr + 1).value, corr.value), corr.vm, corr.n_bits)

      return corr

    # predict output for given input
    def predict(self, input_data):
        output = np.expand_dims(input_data, axis=0)
        for layer in self.layers:
            output = layer.forward(output)

        return encrypted_argmax(output.squeeze())

    # train the network
    def fit(self, x_train, y_train, epochs, miniBatchSize, lrInv):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            epochNumCorrect = 0
            numIter = int(samples/miniBatchSize)

            for j in range(numIter):
                batchNumCorrect = 0
                idxStart = j * miniBatchSize
                idxEnd = idxStart + miniBatchSize

                miniBatchImages = HE_Client.encrypt_matrix(x_train[idxStart:idxEnd])
                miniBarchTargets = HE_Client.encrypt_matrix(y_train[idxStart:idxEnd])

                # forward propagation
                output = miniBatchImages
                
                start_time = time.time()

                for layer in self.layers:
                  output = layer.forward(output)

                end_time = time.time()

                print("End forward batch: " + repr(j))
                print("Computation time: ")
                hours, rem = divmod(end_time-start_time, 3600)
                minutes, seconds = divmod(rem, 60)
                print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
                print("")

                # sumLoss += batchL2Loss(miniBarchTargets, output);
                lossDeltaMat = batchL2LossDelta(miniBarchTargets, output)

#                 dec_out = HE_Client.decrypt_matrix(output)
#                 dec_targets = HE_Client.decrypt_matrix(miniBarchTargets)

                # for r in range(miniBatchSize):
                #     if dec_targets[r].argmax() == dec_out[r].argmax():
                #         batchNumCorrect += 1
                
                start_time = time.time()

                # backward propagation
                for layer in reversed(self.layers):
                    layer.backward(lossDeltaMat, lrInv)

                end_time = time.time()

                print("End backward batch: " + repr(j))
                print("Computation time: ")
                hours, rem = divmod(end_time-start_time, 3600)
                minutes, seconds = divmod(rem, 60)
                print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
                print("")

                epochNumCorrect += batchNumCorrect;

            print("Epoch: " + repr(i))
            print("EpochNumCorrect: " + repr(epochNumCorrect))
            print("Accuracy: " + repr(epochNumCorrect/samples * 100) + " %")
            print("")

In [ ]:
## UPLOAD DFA WEIGHTS
DFAWeights1 = np.load("DFAWeights_L1.npy")
DFAWeights2 = np.load("DFAWeights_L2.npy")

In [ ]:
# Network
net = EncryptedNetwork()
net.add(EncryptedMaxPoolLayer((4, 4), stride=(4, 4)))
net.add(EncryptedFlattenLayer())
net.add(EncryptedFCLayer(16, 4))
net.add(EncryptedFCLayer(4, 2))
net.add(EncryptedFCLayer(2, 3, outLayer=True))

net.layers[2].mDfaWeight = HE_Client.encode_matrix(DFAWeights1)
net.layers[3].mDfaWeight = HE_Client.encode_matrix(DFAWeights2)

In [ ]:
%%time
# training
net.fit(x_train[100:150], y_train[100:150], epochs=2, miniBatchSize=5, lrInv=256)

In [ ]:
weights2 = HE_Client.decrypt_matrix(net.layers[2].weights)
bias2 = HE_Client.decrypt_matrix(net.layers[2].bias)
weights3 = HE_Client.decrypt_matrix(net.layers[3].weights)
bias3 = HE_Client.decrypt_matrix(net.layers[3].bias)
weights4 = HE_Client.decrypt_matrix(net.layers[4].weights)
bias4 = HE_Client.decrypt_matrix(net.layers[4].bias)

In [ ]:
with open("plain_weights_f2.pkl", "wb") as f:
  pickle.dump(weights2, f)
  pickle.dump(bias2, f)
  pickle.dump(weights3, f)
  pickle.dump(bias3, f)
  pickle.dump(weights4, f)
  pickle.dump(bias4, f)

In [ ]:
net.serialize()

with open("encnetMNISTf2.pkl", "wb") as f:
  pickle.dump(net, f)